# Meshes and Boundary Representations

Meshes provide spatial discretization which is the corner stone of any numerical representation.
The shape and the size of cells need to be carefully chosen according to the objective the user wants to reach.

In the particular case of building geological model to investigate its physical response, we need a mesh 
 1. capturing the geometry of large rock heterogeneities,
 1. discretizing elements according to the size of the representative elementary volume of rock properties,
 1. permitting a precise evaluation of the physical response where it is needed, 
 1. presenting cell shapes adapted to method chosen for the numerical computation.

A mesh is a consistent set of joined elements representing a natural "object". Every mesher need a detailled numerical representation of the shape and the topology of the object boundaries (internal & external). This is known as "watertight boundary representation".

This tutorial illustrates the differences between [meshes](https://docs.geode-solutions.com/guides/meshes.html) and [models](https://docs.geode-solutions.com/guides/models.html) (or boundary representations). 

We will explore and use the OpenGeode data model to create and edit 2D boundary representation, named Section.

At the end, you will be able to 
 * create and manipulate meshes and boundary representations,
 * test the validity of boundary representations, 
 * use boundary representation to meshes sealed domains.

In [ ]:
import opengeode as og
import opengeode_io 
import opengeode_inspector as inspector
import geode_common as common
import geode_simplex as simplex


print("Geode module imported!")

# define various utilities functions
import os
def input(path):
  return os.path.join("data/",path) 

def create_folder_if_does_not_exists(path):
  if( not os.path.isdir(path)):
    os.makedirs(path)

def output(path):
  result_path = "results/"
  create_folder_if_does_not_exists(result_path)
  return os.path.join(result_path,path)

## Geometrical impact of remeshing on 2D surface.

This fist part illustrates potential geometric deformation due to unconstrained changes performed during a remeshing algorithm.

The initial model can be loaded from the data folder.

In [ ]:
# load section
section = og.load_section(input("01_single_surface_and_line.og_sctn"))

The section is composed by 
 * a single line represented by a closed edged curve with 4 edges 
 * a single surface represented by a triangulated surface with two triangles.

In [ ]:
# define print section description
def print_section_components(section) : 
    bb =og.BoundingBox2D()
    bb.add_box(section.bounding_box())
    print("The section is centered on (" + 
          str(bb.center().value(0))+" , "+str(bb.center().value(1)) + 
          "). It is a "+str(bb.diagonal().value(0))+"m by "+
          str(bb.diagonal().value(1))+
          "m section with: ")
    print(" * "+ str(section.nb_corners()) +" corners" )
    for corner in section.corners():
        print("  - corner "+ corner.id().string()+" have " + str(corner.mesh().nb_vertices())+" vertices.") 
    print(" * "+ str(section.nb_lines()) +" lines" )
    for line in section.lines():
        print("  - line "+ line.id().string()+" have " + str(line.mesh().nb_edges())+" edges.") 
    print(" * "+ str(section.nb_surfaces()) +" surfaces" )
    for surface in section.surfaces():
        print("  - surface "+ surface.id().string()+" have " + str(surface.mesh().nb_polygons())+" triangles.") 

# print section description
print_section_components(section)

A remeshing of the section will :
 1. modify the number of edges in the edged curve and the number of triangles in the triangulated surface.
 1. preserve the topology of both the line and the surface 

In [ ]:
# set up parameters for homogeneous remeshing
edge_length = 3
cst_metric = common.ConstantMetric2D(edge_length)

# create and return the remeshed section
remeshed_section = simplex.section_simplex_remesh(section,cst_metric)[0]

# print description of the remeshed section
print_section_components(remeshed_section)

The remeshed section can be save in a vtm file format for an inspection in paraview.

In [ ]:
# save section for paraview
og.save_section(remeshed_section, output("01_remeshed_surface.vtm"))
# save section for Vease
og.save_section(remeshed_section, output("01_remeshed_surface.og_sctn"))

<img src="./pictures/surface_remesh.png" alt="image" width="50%" height="auto">

The topological information stored ininital section is not enough to recover a satisfying line and surface geometry after remeshing. 

Several edge_length length values can be investigated. The value need to be strictly positive (0 <) and lower or equal to section extend (<= 10).

Acording to the edge length targetted by the remeshing algorithm the geometry is more or less approximated but the topology is always preserved.

## Create section with the minimal topological information recovert the initial geometry.

Building the following section is a three step workflow:
 1. build model components and their topological relationships,
 1. build the geometry (meshes) of the model components,
 1. build unique vertex identifier to link the geometry and the topology.

![section data model](pictures/section_data_model.png)

In [ ]:
# create an empty section and its builder 
section = og.Section()
builder = og.SectionBuilder(section)

print_section_components(section)

The next step is to create section components with their consitent topological relations
 * 4 corners (a corner should be a line boundary)
 * 4 lines (a line is a surface boundary and is incident corners)
 * 1 surface (a surface is incident to a line)

![section data model component](pictures/section_data_model_component.png)

Note that: 
 * line can also be internal to a surface
 * in 3D a surface is a boundary of block 
 * in 3D a line and a surface can be internal to a block

In [ ]:
# create a line bounded by two corners
def add_line(section, builder, corner_id0, corner_id1):
    line_id = builder.add_line()
    line = section.line(line_id)
    builder.add_corner_line_boundary_relationship(section.corner(corner_id0), line)
    builder.add_corner_line_boundary_relationship(section.corner(corner_id1), line)
    return line_id

# create a surface bounded by lines
def add_surface(section, builder, line_ids):
    surface_id = builder.add_surface()
    surface = section.surface(surface_id)
    for line_id in line_ids:
        builder.add_line_surface_boundary_relationship(section.line(line_id), surface)
    return surface_id

# create corner
corner1 = builder.add_corner()
corner2 = builder.add_corner()
corner3 = builder.add_corner()
corner4 = builder.add_corner()

# create line
line1 =  add_line(section,builder,corner1,corner2)
line2 =  add_line(section,builder,corner2,corner3)
line3 =  add_line(section,builder,corner3,corner4)
line4 =  add_line(section,builder,corner4,corner1)

# create surface
surface = add_surface(section, builder, [line1,line2,line3,line4])

#print section description
print_section_components(section)

WARNING: each execution of the previous code will create new model components!!

Each model component will now be associated to its geometrical representation:
 * one vertex for each corner
 * one edged curve for each line
 * one triangulated surface for each surface 

![section data mesh component](pictures/section_data_mesh_component.png)

In [ ]:
def create_line_mesh(builder, line_id, point0, point1):
    line_builder = builder.line_mesh_builder(line_id)
    line_builder.create_point(point0)
    line_builder.create_point(point1)
    line_builder.create_edge_with_vertices(0,1)

def create_surface_mesh(section, builder, surface_id, points):
    og.convert_section_surface_meshes_into_triangulated_surfaces(section)
    surface_builder = builder.surface_mesh_builder(surface_id)
    for pt in points:
        surface_builder.create_point(pt)
    surface_builder.create_polygon([0,1,2])
    surface_builder.create_polygon([0,2,3])
    surface_builder.compute_polygon_adjacencies()

# create corner vertex
pt1 = og.Point2D([0.,0.])
pt2 = og.Point2D([10.,0.])
pt3 = og.Point2D([10.,10.])
pt4 = og.Point2D([0.,10.])

# create corner mesh
builder.corner_mesh_builder(corner1).create_point(pt1)
builder.corner_mesh_builder(corner2).create_point(pt2)
builder.corner_mesh_builder(corner3).create_point(pt3)
builder.corner_mesh_builder(corner4).create_point(pt4)

# create line mesh
create_line_mesh(builder, line1, pt1, pt2)
create_line_mesh(builder, line2, pt2, pt3)
create_line_mesh(builder, line3, pt3, pt4)
create_line_mesh(builder, line4, pt4, pt1)

# create surface mesh
create_surface_mesh(section, builder, surface, [pt1, pt2, pt3, pt4])

print_section_components(section)

WARNING: each execution of the previous code will create new mesh components!! In this case you will have colocated points and elements.

The last step is to setup unique vertex identifier to ensure that a geometrical update made on any model component can be consistently propagated to others.

![section data unique vertex](pictures/section_data_unique_vertex.png)

In [ ]:
# create lower left unique vertex
model_vertex_id1 = builder.create_unique_vertex()
builder.set_unique_vertex(og.ComponentMeshVertex(section.corner(corner1).component_id(), 0), model_vertex_id1)
builder.set_unique_vertex(og.ComponentMeshVertex(section.line(line1).component_id(), 0), model_vertex_id1)
builder.set_unique_vertex(og.ComponentMeshVertex(section.line(line4).component_id(), 1), model_vertex_id1)
builder.set_unique_vertex(og.ComponentMeshVertex(section.surface(surface).component_id(), 0), model_vertex_id1)

# create upper left unique vertex
model_vertex_id2 = builder.create_unique_vertex()
builder.set_unique_vertex(og.ComponentMeshVertex(section.corner(corner2).component_id(), 0), model_vertex_id2)
builder.set_unique_vertex(og.ComponentMeshVertex(section.line(line2).component_id(), 0), model_vertex_id2)
builder.set_unique_vertex(og.ComponentMeshVertex(section.line(line1).component_id(), 1), model_vertex_id2)
builder.set_unique_vertex(og.ComponentMeshVertex(section.surface(surface).component_id(), 1), model_vertex_id2)

# create upper right unique vertex
model_vertex_id3 = builder.create_unique_vertex()
builder.set_unique_vertex(og.ComponentMeshVertex(section.corner(corner3).component_id(), 0), model_vertex_id3)
builder.set_unique_vertex(og.ComponentMeshVertex(section.line(line3).component_id(), 0), model_vertex_id3)
builder.set_unique_vertex(og.ComponentMeshVertex(section.line(line2).component_id(), 1), model_vertex_id3)
builder.set_unique_vertex(og.ComponentMeshVertex(section.surface(surface).component_id(), 2), model_vertex_id3)

# create lower right unique vertex
model_vertex_id4 = builder.create_unique_vertex()
builder.set_unique_vertex(og.ComponentMeshVertex(section.corner(corner4).component_id(), 0), model_vertex_id4)
builder.set_unique_vertex(og.ComponentMeshVertex(section.line(line4).component_id(), 0), model_vertex_id4)
builder.set_unique_vertex(og.ComponentMeshVertex(section.line(line3).component_id(), 1), model_vertex_id4)
builder.set_unique_vertex(og.ComponentMeshVertex(section.surface(surface).component_id(), 3), model_vertex_id4)

The OpenGeode ecosystem provides inspection tools to check the topological and geometrical validity of boundary representations and meshes.

It is also avaible as [free cloud service](https://geode-solutions.com/tools).

In [ ]:
#inspector
section_inspector = inspector.SectionInspector(section)
result = section_inspector.inspect_section()
print(result.string())

# save 2D BRep for a potential inspection online
og.save_section(section, output("01_section_valid.og_sctn"))

Once you created a valid section, you can perform an homogeneous remeshing with various edge_length values and visualize results with paraview.

In [ ]:

# set up parameters for homogeneous remeshing
edge_length = 3
cst_metric = common.ConstantMetric2D(edge_length)

# create and return the remeshed section
remeshed_section = simplex.section_simplex_remesh(section,cst_metric)[0]
og.save_section(remeshed_section, output("01_remeshed_section_valid.og_sctn"))

# print description of the remeshed section
print_section_components(remeshed_section)

Similar tutorial can be done in 3d to build a BRep.

Have a look [here](https://github.com/Geode-solutions/OpenGeode/blob/master/examples/layer_cake.ipynb) to learn how to build a layer cake BRep.